In [ ]:
# authenticator.py

import csv
import time
import logging
import psutil
import os
import uuid
from flask import Flask, request, jsonify

app = Flask(__name__)

class Authenticator:
    def __init__(self, csv_file='entity_authentication.csv', pa=None):
        self.csv_file = csv_file
        self.entity_data = {}  # Stores authentication data for each entity
        self.pa = pa  # Reference to PolicyAdministration (not used in this context)
        self.load_data()
        self.setup_logging()

    def setup_logging(self):
        # Configure the logging module
        logging.basicConfig(
            filename='authenticator.log',
            level=logging.INFO,
            format='%(asctime)s %(levelname)s: %(message)s'
        )

    def load_data(self):
        """
        Loads authentication data from the CSV file into a dictionary.
        """
        logging.info("Read csv")
        try:
            with open(self.csv_file, 'r', encoding='utf-8-sig') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    entity_id = row['entity_id']
                    self.entity_data[entity_id] = {
                        'certificate_validation_time': row.get('certificate_validation_time', ''),
                        'installer_signature': row.get('installer_signature', ''),
                        'approval_signature': row.get('approval_signature', ''),
                        'authenticated_status': row.get('authenticated_status', 'unauthenticated').strip().lower()
                    }
                print(f"Authenticator: Loaded data for {len(self.entity_data)} entities.")
        except FileNotFoundError:
            print(f"Authenticator Error: CSV file '{self.csv_file}' not found.")
            self.entity_data = {}
        except Exception as e:
            print(f"Authenticator Error: {e}")
            self.entity_data = {}

    def authenticate(self, entity_id, request_data, request_id):
        """
        Authenticates the entity based on the loaded data and policies.

        Args:
            entity_id (str): The ID of the entity requesting access.
            request_data (dict): Additional request data.
            request_id (str): Unique identifier for the request.

        Returns:
            str: 'authenticated' or 'unauthenticated'
        """
        # Start CPU and timing measurements
        process = psutil.Process(os.getpid())
        cpu_times_start = process.cpu_times()
        start_time = time.time()

        if entity_id == 'user':
            auth_status = self.authenticate_user(request_data)
        else:
            auth_status = self.authenticate_device(entity_id)

        # End CPU and timing measurements
        end_time = time.time()
        cpu_times_end = process.cpu_times()

        # Calculate CPU time and processing time
        user_cpu_time = cpu_times_end.user - cpu_times_start.user
        system_cpu_time = cpu_times_end.system - cpu_times_start.system
        total_cpu_time = user_cpu_time + system_cpu_time
        processing_time = end_time - start_time

        # Log the performance data
        logging.info(
            f"Request ID: {request_id} | Entity ID: {entity_id} | Authentication Status: {auth_status} | "
            f"Processing Time: {processing_time:.6f}s | CPU Time: User={user_cpu_time:.6f}s System={system_cpu_time:.6f}s Total={total_cpu_time:.6f}s"
        )

        return auth_status

    def authenticate_user(self, request_data):
        """
        Authenticates the user by verifying username, password, and IP address.

        Args:
            request_data (dict): Request data containing 'username', 'password', and 'ip_address'.

        Returns:
            str: 'authenticated' or 'unauthenticated'
        """
        # Retrieve expected values (hardcoded for this example)
        expected_ip = '192.52.36.185'
        authentication_required = True

        provided_username = request_data.get('username', '')
        provided_password = request_data.get('password', '')
        provided_ip = request_data.get('ip_address', '')

        # Verify credentials and IP address
        if (self.verify_credentials(provided_username, provided_password) and
            provided_ip == expected_ip):
            return 'authenticated'
        else:
            print("Authenticator: User authentication failed.")
            return 'unauthenticated'

    def authenticate_device(self, entity_id):
        """
        Authenticates a device based on the loaded CSV data.

        Args:
            entity_id (str): The ID of the device.

        Returns:
            str: 'authenticated' or 'unauthenticated'
        """
        # Check if entity_id exists in data
        if entity_id in self.entity_data:
            auth_status = self.entity_data[entity_id]['authenticated_status']
            if auth_status == 'authenticated':
                return 'authenticated'
            else:
                print(f"Authenticator: Device '{entity_id}' is unauthenticated.")
                return 'unauthenticated'
        else:
            # Entity ID not found
            print(f"Authenticator: Device ID '{entity_id}' not found in authentication data.")
            return 'unauthenticated'

    def verify_credentials(self, username, password):
        """
        Verifies the user's credentials.

        Args:
            username (str): The provided username.
            password (str): The provided password.

        Returns:
            bool: True if credentials are valid, False otherwise.
        """
        # Implement credential verification logic
        # For this example, assume 'valid_user' and 'valid_password' are correct
        if username == 'valid_user' and password == 'valid_password':
            return True
        else:
            print("Authenticator: Invalid username or password.")
            return False

# Create an instance of Authenticator
authenticator = Authenticator()

@app.route('/authenticate', methods=['POST'])
def authenticate_endpoint():
    """
    API endpoint to authenticate an entity.
    Expects JSON data with 'entity_id', 'request_data', and 'request_id'.
    """
    try:
        json_data = request.get_json()
        entity_id = json_data.get('entity_id')
        request_data = json_data.get('request_data', {})
        request_id = json_data.get('request_id', str(uuid.uuid4()))

        auth_status = authenticator.authenticate(entity_id, request_data, request_id)

        response = {
            'request_id': request_id,
            'entity_id': entity_id,
            'authentication_status': auth_status
        }

        return jsonify(response), 200
    except Exception as e:
        logging.error(f"Error in authenticate_endpoint: {e}")
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # Run the Flask app on all interfaces, port 5001
    app.run(host='0.0.0.0', port=5001)